In [2]:
!unzip /content/drive/MyDrive/train.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: seg_train/mountain/7539.jpg  
  inflating: seg_train/mountain/7551.jpg  
  inflating: seg_train/mountain/7560.jpg  
  inflating: seg_train/mountain/7565.jpg  
  inflating: seg_train/mountain/7578.jpg  
  inflating: seg_train/mountain/7581.jpg  
  inflating: seg_train/mountain/7586.jpg  
  inflating: seg_train/mountain/7647.jpg  
  inflating: seg_train/mountain/7652.jpg  
  inflating: seg_train/mountain/7654.jpg  
  inflating: seg_train/mountain/7662.jpg  
  inflating: seg_train/mountain/767.jpg  
  inflating: seg_train/mountain/7672.jpg  
  inflating: seg_train/mountain/7679.jpg  
  inflating: seg_train/mountain/7681.jpg  
  inflating: seg_train/mountain/7693.jpg  
  inflating: seg_train/mountain/7695.jpg  
  inflating: seg_train/mountain/7698.jpg  
  inflating: seg_train/mountain/7700.jpg  
  inflating: seg_train/mountain/771.jpg  
  inflating: seg_train/mountain/7715.jpg  
  inflating: seg_train/mountain/7744.jpg  

In [3]:
!unzip /content/drive/MyDrive/test.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: test/1750.jpg           
  inflating: __MACOSX/test/._1750.jpg  
  inflating: test/1988.jpg           
  inflating: __MACOSX/test/._1988.jpg  
  inflating: test/1817.jpg           
  inflating: __MACOSX/test/._1817.jpg  
  inflating: test/429.jpg            
  inflating: __MACOSX/test/._429.jpg  
  inflating: test/1803.jpg           
  inflating: __MACOSX/test/._1803.jpg  
  inflating: test/367.jpg            
  inflating: __MACOSX/test/._367.jpg  
  inflating: test/1195.jpg           
  inflating: __MACOSX/test/._1195.jpg  
  inflating: test/401.jpg            
  inflating: __MACOSX/test/._401.jpg  
  inflating: test/2488.jpg           
  inflating: __MACOSX/test/._2488.jpg  
  inflating: test/415.jpg            
  inflating: __MACOSX/test/._415.jpg  
  inflating: test/1181.jpg           
  inflating: __MACOSX/test/._1181.jpg  
  inflating: test/373.jpg            
  inflating: __MACOSX/test/._373.jpg  
  inflating:

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3

print("Setup Complete!")

Setup Complete!


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train_DIR = "/content/seg_train/"

train_datagen = ImageDataGenerator( rescale = 1.0/255,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          vertical_flip=True,
                                          fill_mode='nearest')


train_generator = train_datagen.flow_from_directory(train_DIR,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

test_DIR = "/content/test/"
validation_datagen = ImageDataGenerator(rescale = 1.0/255)


validation_generator = validation_datagen.flow_from_directory(test_DIR,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

Found 14034 images belonging to 6 classes.
Found 0 images belonging to 0 classes.


In [7]:
inceptionV3 = InceptionV3(include_top= False, input_shape=(150,150,3))

for layer in inceptionV3.layers:
	layer.trainable = False

87910968/87910968 [==============================] - 3s 0us/step


In [8]:
last_layer = inceptionV3.get_layer('mixed9')

print('last layer output shape: ', last_layer.output_shape)

last_output = last_layer.output

last layer output shape:  (None, 3, 3, 2048)


In [9]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(units = 1024, activation = tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense  (6, activation = tf.nn.softmax)(x)

model = tf.keras.Model( inceptionV3.input, x)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=1,
                                            verbose=1,
                                            factor=0.25,
                                            min_lr=0.000003)

model.compile(loss = 'categorical_crossentropy', optimizer= tf.keras.optimizers.Adam(), metrics=['acc'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [13]:
history = model.fit(train_generator,
                    epochs = 1,
                    verbose = 1,
                   callbacks=[learning_rate_reduction])

439/439 [==============================] - ETA: 0s - loss: 0.3472 - acc: 0.8730

439/439 [==============================] - 81s 184ms/step - loss: 0.3472 - acc: 0.8730 - lr: 0.0010


In [14]:
path = "/content/test"

In [15]:
import numpy as np
import pandas as pd

In [18]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    path ,
    labels=None,
    shuffle=False,
    image_size=(150, 150 ),
    batch_size=128)

y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred,axis=1)

sample = pd.read_csv('/content/drive/MyDrive/sample.csv')
submit = sample.copy(deep=True)
submit.label = y_pred
file_path = pd.Series(test_dataset.file_paths)
new_index = file_path.str.split('/', expand=True).iloc[:,-1].str.split('.', expand=True)[0].astype('int')
submit.id = new_index
submit.sort_values(by='id',inplace=True)
submit.to_csv(path_or_buf='./filename.csv', index=False)

Found 3000 files belonging to 1 classes.
24/24 [==============================] - 4s 162ms/step
